# atomman.lammps.normalize(system)

- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2017-04-19

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The normalize function returns a new System identical to an existing one but with Box vectors and atomic positions transformed to be compatible with LAMMPS. All atom positions are adjusted to be within the box ([wrapped](atomman.System.wrap.ipynb)) and System.box's vectors transformed to be compatible. This function is called by the [System.normalize](atomman.System.normalize.ipynb) method with style='lammps'.

Arguments:

- __system__ -- a [System](atomman.System) to transform into a LAMMPS compatible System. 

In LAMMPS, the Box vectors must conform to the following rules:

    avect = [ lx, 0.0, 0.0]
    bvect = [ xy,  ly, 0.0]
    cvect = [ xz,  yz,  lz]
    avect, bvect, cvect are right-handed.
    
__WARNING!__ Tilt factors are not tested for compliance with LAMMPS. If large angles/tilt factors are present, the command "box tilt large" may be required in the LAMMPS input script.

The underlying code can be found in [atomman/lammps/normalize.py](../../atomman/lammps/normalize.py).

## Demonstration

In [1]:
#Standard libraries
from __future__ import print_function
from copy import deepcopy

#http://www.numpy.org/
import numpy as np   

#https://github.com/usnistgov/atomman
import atomman as am 
import atomman.lammps as lmp

### Normalize a messy random System

Create a messy test system with atoms outside bounds and random box vectors

In [2]:
atoms = am.Atoms(natoms=10, prop={'atype':1, 'pos':10*np.random.random((10,3))})
box = am.Box(vects=5*np.random.random((3,3)), origin=(5, 5, 5))

system = am.System(atoms=atoms, box=box)
print(system)

avect =  [ 0.453,  4.099,  0.809]
bvect =  [ 4.732,  4.759,  3.333]
cvect =  [ 4.441,  4.816,  4.733]
origin = [ 5.000,  5.000,  5.000]
natoms = 10
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   7.727 |   0.799 |   8.572
      1 |       1 |   5.507 |   7.011 |   7.823
      2 |       1 |   3.739 |   2.585 |   0.072
      3 |       1 |   1.896 |   6.415 |   1.646
      4 |       1 |   7.175 |   8.376 |   8.316
      5 |       1 |   5.546 |   5.167 |   9.580
      6 |       1 |   5.604 |   2.060 |   5.635
      7 |       1 |   8.069 |   8.857 |   5.350
      8 |       1 |   4.671 |   6.030 |   3.964
      9 |       1 |   2.053 |   7.198 |   7.285


Create a new normalized system

In [3]:
lammps_system = lmp.normalize(system)
print(lammps_system)

avect =  [ 4.202,  0.000,  0.000]
bvect =  [ 5.793,  4.752,  0.000]
cvect =  [-6.087, -5.143,  1.344]
origin = [ 0.000,  0.000,  0.000]
natoms = 10
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   3.109 |  -0.721 |   0.456
      1 |       1 |   1.971 |   0.597 |   0.734
      2 |       1 |   1.350 |  -2.155 |   0.666
      3 |       1 |   2.285 |   0.629 |   0.191
      4 |       1 |  -2.216 |  -2.739 |   1.287
      5 |       1 |   4.423 |   1.406 |   0.367
      6 |       1 |   5.430 |   0.980 |   0.751
      7 |       1 |   4.454 |   2.483 |   0.310
      8 |       1 |   0.769 |  -0.961 |   0.852
      9 |       1 |  -2.819 |  -3.065 |   0.867


### Check that the cell parameters of the original and normalized systems match

Lattice parameters match, and lattice angles will either match or old+new values add up to 180.

In [4]:
print('a =',     system.box.a)
print('b =',     system.box.b)
print('c =',     system.box.c)
print('alpha =', system.box.alpha)
print('beta =',  system.box.beta)
print('gamma =', system.box.gamma)

a = 4.2022469878
b = 7.49291906406
c = 8.08158703523
alpha = 9.6081073828
beta = 41.1271397532
gamma = 39.3626610002


In [5]:
print('a =',     lammps_system.box.a)
print('b =',     lammps_system.box.b)
print('c =',     lammps_system.box.c)
print('alpha =', lammps_system.box.alpha)
print('beta =',  lammps_system.box.beta)
print('gamma =', lammps_system.box.gamma)

a = 4.2022469878
b = 7.49291906406
c = 8.08158703523
alpha = 170.391892617
beta = 138.872860247
gamma = 39.3626610002


### Check that relative atom positions are consistent and in the Box

The original System has atoms outide the System's Box (scaled positions not between 0 and 1)

In [6]:
print(system.atoms_prop(key='pos', scale=True))

[[-2.06563697 -0.78428488  1.66050582]
 [ 0.27203437 -1.28327662  1.45387084]
 [-0.07489802  2.08244061 -2.49522468]
 [ 1.3552108   0.28574628 -1.14179697]
 [ 0.22575103 -0.54046121  1.04273572]
 [-0.36725021 -2.40874706  2.7271223 ]
 [-1.01617336 -0.18918711  0.44127275]
 [ 0.32964756  1.77197157 -1.23050875]
 [ 0.43450363  0.48384086 -0.63398745]
 [ 1.19041172 -2.94699695  2.3550335 ]]


Wrapping the system corrects this

In [7]:
system.wrap()
print(system.atoms_prop(key='pos', scale=True))

[[ 0.93436303  0.21571512  0.66050582]
 [ 0.27203437  0.71672338  0.45387084]
 [ 0.92510198  0.08244061  0.50477532]
 [ 0.3552108   0.28574628  0.85820303]
 [ 0.22575103  0.45953879  0.04273572]
 [ 0.63274979  0.59125294  0.7271223 ]
 [ 0.98382664  0.81081289  0.44127275]
 [ 0.32964756  0.77197157  0.76949125]
 [ 0.43450363  0.48384086  0.36601255]
 [ 0.19041172  0.05300305  0.3550335 ]]


The lammps normalized System has atomic positions consistent with the wrapped System. The relative z coordinates may be flipped if the original box vectors were left-handed.

In [8]:
print(lammps_system.atoms_prop(key='pos', scale=True))

[[ 0.93436303  0.21571512  0.33949418]
 [ 0.27203437  0.71672338  0.54612916]
 [ 0.92510198  0.08244061  0.49522468]
 [ 0.3552108   0.28574628  0.14179697]
 [ 0.22575103  0.45953879  0.95726428]
 [ 0.63274979  0.59125294  0.2728777 ]
 [ 0.98382664  0.81081289  0.55872725]
 [ 0.32964756  0.77197157  0.23050875]
 [ 0.43450363  0.48384086  0.63398745]
 [ 0.19041172  0.05300305  0.6449665 ]]


- - -
__Docs Navigation:__

Tutorial:

1. [Basics](../tutorial/1 Basics.ipynb)

2. [LAMMPS Functionality](../tutorial/2 LAMMPS Functionality.ipynb)

3. [Defect Generation and Evaluation](../tutorial/3 Defect Generation and Evaluation.ipynb)


Reference:

- [atomman](../reference/atomman.ipynb)

- [atomman.convert](../reference/atomman.convert.ipynb)

- [atomman.defect](../reference/atomman.defect.ipynb)

- [atomman.lammps](../reference/atomman.lammps.ipynb)

- [atomman.tools](../reference/atomman.tools.ipynb)

- [atomman.unitconvert](../reference/atomman.unitconvert.ipynb)